In [15]:
import torch
import torchvision
import math
import pandas as pd
import numpy as np
import gc
from tqdm import tqdm_notebook
from torch.utils.tensorboard import SummaryWriter
from torch import nn


from GWD_data import WheatDataset
from metric import calculate_image_precision
from utils import collate_fn, plot_boxes, get_model_name, format_prediction_string, remove_blanks, is_contain_blanks
from config import config


In [16]:
conf = config()
torch.random.manual_seed(5)
writer = SummaryWriter("runs/May11_23-57-42_DESKTOP-ELPLUSQ") #


WD_Train = WheatDataset(conf)
WD_Valid = WheatDataset(conf, train=False)
WD_Train_Loader = torch.utils.data.DataLoader(WD_Train, batch_size=conf.BATH_SIZE, shuffle=True, collate_fn=collate_fn)
WD_Valid_Loader = torch.utils.data.DataLoader(WD_Valid, batch_size=conf.BATH_SIZE, shuffle=True, collate_fn=collate_fn)

In [17]:
class ModelWithLoss(nn.Module):
    def __init__(self, model, debug=False):
        super().__init__()
        self.criterion = FocalLoss()
        self.model = model
        self.debug = debug

    def forward(self, imgs, annotations, obj_list=None):
        _, regression, classification, anchors = self.model(imgs)
        if self.debug:
            cls_loss, reg_loss = self.criterion(classification, regression, anchors, annotations,
                                                imgs=imgs, obj_list=obj_list)
        else:
            cls_loss, reg_loss = self.criterion(classification, regression, anchors, annotations)
        return cls_loss, reg_loss

In [20]:
import torch.nn.functional as F
from EfficientDet.backbone import EfficientDetBackbone
from EfficientDet.efficientdet.loss import FocalLoss

EPOCH = 0 #int(model_path.split('/')[-1].split('_')[2])+1 if model_path else 0

#num_classes=1
compound_coef=2 #default level
#num_classes=num_classes
anchor_ratios = [(1.0, 1.0), (1.4, 0.7), (0.7, 1.4)]
anchor_scales = [2 ** 0, 2 ** (1.0 / 3.0), 2 ** (2.0 / 3.0)]

GWD_effdet = EfficientDetBackbone( compound_coef=compound_coef,num_classes=1, ratios=anchor_ratios, scales=anchor_scales)

state_dict = torch.load(f'./EfficientDet/efficientdet-d{compound_coef}.pth')


try:
    ret = GWD_effdet.load_state_dict(state_dict)  #Strict=False ignores non matching keys
except RuntimeError as e:
    print(f'[Warning] Ignoring {e}')
    print('[Warning] Don\'t panic if you see this, this might be because you load a pretrained weights with different number of classes. The rest of the weights should be loaded already.')


debug=False
GWD_effdet = ModelWithLoss(GWD_effdet, debug=debug)


GWD_effdet.to(conf.DEVICE)
params = [p for p in GWD_effdet.parameters() if p.requires_grad]
optimizer =  torch.optim.AdamW(model.parameters(), 0.00001)    #These very the default values in source code
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, verbose=True)

[Warning] Ignoring Error(s) in loading state_dict for EfficientDetBackbone:
	size mismatch for classifier.header.pointwise_conv.conv.weight: copying a param with shape torch.Size([810, 112, 1, 1]) from checkpoint, the shape in current model is torch.Size([9, 112, 1, 1]).
	size mismatch for classifier.header.pointwise_conv.conv.bias: copying a param with shape torch.Size([810]) from checkpoint, the shape in current model is torch.Size([9]).
[Warning] Don't panic if you see this, this might be because you load a pretrained weights with different number of classes. The rest of the weights should be loaded already.


In [27]:
_iter=len(WD_Train_Loader)*(EPOCH)
clip_max_norm=0.1  #Default value in their code

for i in tqdm_notebook(range(EPOCH, 50, 1)):
    
    _epoch_loss=0
    _ = GWD_effdet.train()
    
    for images, targets in tqdm_notebook(WD_Train_Loader):

        images = [torch.tensor(image, dtype = torch.float32).to(conf.DEVICE) for image in images]
        targets = [{k: torch.tensor(v).to(conf.DEVICE) for k, v in target.items()} for target in targets]
        
        images = np.array(images)
        print(images.shape)
        
        optimizer.zero_grad()
        cls_loss, reg_loss = GWD_effdet(images, targets, obj_list=['wheat'])
        cls_loss = cls_loss.mean()
        reg_loss = reg_loss.mean()

        loss = cls_loss + reg_loss
        if loss == 0 or not torch.isfinite(loss):
            continue

        loss.backward()
        # torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()

        progress_bar.set_description('Step: {}. Epoch: {}/{}. Iteration: {}/{}. Cls loss: {:.5f}. Reg loss: {:.5f}. Total loss: {:.5f}'.format(
                    step, epoch, opt.num_epochs, iter + 1, num_iter_per_epoch, cls_loss.item(), reg_loss.item(), loss.item()))
        writer.add_scalars('Loss', {'train': loss}, step)
        writer.add_scalars('Regression_loss', {'train': reg_loss}, step)
        writer.add_scalars('Classfication_loss', {'train': cls_loss}, step)

        # log learning_rate
        current_lr = optimizer.param_groups[0]['lr']
        writer.add_scalar('learning_rate', current_lr, step)
         
        _iter+=1
        break
    
    scheduler.step(np.mean(epoch_loss))
    
    print('Saving model at epoch '+str(i)+', step '+str(_iter))
    torch.save(GWD_detr_Model, 'model-detr/'+get_model_name(i, 0, _epoch_loss))
    torch.cuda.empty_cache()
    gc.collect()
    break

<ipython-input-27-02b7774f0efa>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(EPOCH, 50, 1)):


<ipython-input-27-02b7774f0efa>:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for images, targets in tqdm_notebook(WD_Train_Loader):


ValueError: only one element tensors can be converted to Python scalars